In [1]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict

from random import *

In [2]:
vertex_set = set();
vertex_dict = {}
vertex_followlist ={}
sink_dict = {};
sink_followedlist = defaultdict(list)

with open("train.txt") as trainfile:
    for i, line in enumerate(trainfile):
        line_list = [int(k) for k in line[:-1].split("\t")];
        vertex_dict[line_list[0]]=len(line_list)-1
        vertex_set.add(line_list[0]);
        vertex_followlist[line_list[0]]= line_list[1:]
        for s in line_list[1:]:
            sink_followedlist[s].append(line_list[0])
            if s in sink_dict:
                sink_dict[s] += 1;
            else:
                sink_dict[s] = 1;
        if i % 1000 == 0:
            print(i);

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [3]:
print(sink_dict[4867135]);
print(len(vertex_set));

1
20000


In [4]:
new_sink_dict = {};
threshold = 20;
for k in sink_dict:
    if sink_dict[k] >= threshold:
        new_sink_dict[k] = sink_dict[k];
        
new_sink_set = set(new_sink_dict);
print(len(new_sink_set))

222237


In [5]:
test_vertex_and_sink_set = set();

with open("test-public.txt") as testfile:
    for i, line in enumerate(testfile):
        if i == 0:
            continue;
        line_list = [int(k) for k in line[:-1].split("\t")];
        for s in line_list:
            test_vertex_and_sink_set.add(s);
print(len(test_vertex_and_sink_set));

total_set = test_vertex_and_sink_set.union(new_sink_set).union(vertex_set);
print(len(total_set));

5947
232230


In [6]:
total_dict = {};
total_list = []
for i, p in enumerate(total_set):
    total_dict[p] = i
    total_list.append(p)

In [7]:
max_neighbors = 1000

import numpy as np
total_array = np.array(total_list);

pairs = [];
true_pairs = [];
"""
"""

with open("train.txt") as trainfile:
    for i, line in enumerate(trainfile):
        line_list = [int(k) for k in line[:-1].split("\t")];
        v = line_list[0];
        ranking = [-sink_dict[k] for k in line_list[1:]];
        sorting = np.argsort(ranking);
        filtered_linelist = np.array(line_list[1:])[sorting];
        for s in filtered_linelist[1:max_neighbors]:
            if s in total_set:
                pairs.append([total_dict[v], total_dict[s]]);
        if i % 1000 == 0:
            print(i);

test_pairs = [];
with open("test-public.txt") as testfile:
    for i, line in enumerate(testfile):
        if i == 0:
            continue;
        line_list = [int(k) for k in line[:-1].split("\t")];
        test_pairs.append([line_list[0], total_dict[line_list[1]], total_dict[line_list[2]]]);
        
correspondence = total_array

a=0
for i in pairs:
    true_pairs.append([correspondence[i[0]],correspondence[i[1]]])


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [10]:
print (true_pairs[0])
print (len(test_pairs))

[4066935, 848620]
2000


In [11]:
def findneighbor(a,b):
    import math
    """
    c is common friend
    1.a→c→b
    2.a→c←b
    3.a←c→b
    4.a←c←b
    5. Jaccard's coeffcient common friends/total friends
    6. Adamic/Adar 
    """
    tempA,tempB = [],[]
    count6=0
    count7=0
    
    if a in vertex_followlist:
        tempA = vertex_followlist[a]
    if b in vertex_followlist:
        tempB = vertex_followlist[b]
    count1,count2,count3,count4,count5,count6 = 0,0,0,0,0,0

    
    if tempA !=None:
        for i in tempA:
            if i in vertex_followlist:
                if b in vertex_followlist[i]:
                    count1+=1
    
    count2 = len(set(tempA).intersection(tempB))
    if a in sink_followedlist and b in sink_followedlist:
        count3 = len(set(sink_followedlist[a]).intersection(sink_followedlist[b]))
        totalA = set().union(sink_followedlist[a],tempA)
        totalB = set().union(sink_followedlist[b],tempB)
        commonFriends = list(set(totalA).intersection(totalB))
        for commonFriend in commonFriends:
            if commonFriend in vertex_dict:
                if commonFriend in sink_dict:
                    freq_comfri=vertex_dict[commonFriend]+sink_dict[commonFriend]
                else:
                    freq_comfri=vertex_dict[commonFriend]
            else: 
                if commonFriend in sink_dict:
                    freq_comfri=sink_dict[commonFriend]
                else:
                    freq_comfri=1
            count6+=1/math.log(freq_comfri)
            
        total_AB = len(totalA.union(totalB))
        """"""
#         "BFS"       
#         for x in list(totalA)
#             if b in list(totalA):
#                 count7=1
#             else:
#                 x_neighbour_list=list(set(vertex_followlist[x]).union(sink_followedlist[x]))
#                 if b in x_neighbour_list:
#                     "here should be a BFS to calculate the graph distance"        
        """"""
   
    for i in tempB:
        if i in vertex_followlist:
            if a in vertex_followlist[i]:
                count4+=1
                
#     commonFriends = count1+count2+count3+count4
    count5 = len(commonFriends)/total_AB
    
    return (count1,count2,count3,count4,count5,count6)
findneighbor(933347,1131654)

(18, 0, 18, 0, 0.001990049751243781, 1.8320899639845865)

In [12]:
def countFollowingFollower(a,b):
    """return 
    1. a following count 
    2. a followed count
    3. b following count
    4. b followed count
    5. a following/a followed
    6. b following/b followed
    7. if b following a
    8. if a following b (y=label)

    """
    count1,count2,count3,count4,count5,count6,count7,count8=0,0,0,0,0,0,0,0
    if a in vertex_dict:
        count1 = vertex_dict[a]
    if b in vertex_dict:
        count3 = vertex_dict[b]
    if a in sink_dict:
        count2 = sink_dict[a]
    if b in sink_dict:
        count4 = sink_dict[b]
    if count2!=0:
        count5 = count1/count2
    else:
        count5 = count1
    if count4!=0:
        count6 = count3/count4
    else:
        count6 = count2
    if b in vertex_followlist:    
        if a in vertex_followlist[b]:
            count7=1
    if a in vertex_followlist:    
        if b in vertex_followlist[a]:
            count8=1
    return (count1,count2,count3,count4,count5,count6,count7,count8)
#     return (count1,count2,count3,count4,count5,count6)

"""
count8  if a follow b

"""
countFollowingFollower(1406432,2481036)

(84, 16, 14, 24, 5.25, 0.5833333333333334, 0, 0)

In [13]:
def distance(a,b):
    vertex_followlist
    

In [ ]:
input_train = []
count=0
for i in true_pairs:
    a=countFollowingFollower(i[0],i[1])
    b=findneighbor(i[0],i[1])
    count+=1
    input_train.append([b[0],b[1],b[2],b[3],a[0],a[1],a[2],a[3],a[4],a[5],0,0])
    #input_train.append([a[0],a[1],a[2],a[3],a[4],a[5],0,0])
    if count % 10 == 0:
        print(count);
    
print (len(input_train))

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420


In [ ]:
sink_sum = set(sink_dict)
all_set = sink_sum.union(vertex_set)
sum_ = len(all_set)-1
all_set_list = list(all_set)
false_pairs = []
for i in range (5173000):
    random1 = randint(0, sum)
    random2 = randint(0,sum)
    false_pairs.append([all_set_list[random1],all_set_list[random2]])


In [ ]:
count=0    
for i in false_pairs:
    a=countFollowingFollower(i[0],i[1])
    b=findneighbor(i[0],i[1])
    count+=1
    if count % 100000 == 0:
        print(count);
    
    input_train.append([b[0],b[1],b[2],b[3],b[4],b[5],a[0],a[1],a[2],a[3],a[4],a[5],a[6],a[7]])
    
print (len(input_train))
    
niubi = np.array(input_train)